In [28]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [29]:
url = 'https://www.ggpolice.go.kr/main/html.do?menu=HC44'
result = requests.get(url)
soup = BeautifulSoup(result.text)

In [30]:
lis = soup.select('#depth3 > div > div.wrap_in.float_wrap > div > section > div > nav > ul > li')
len(lis)

31

In [31]:
lis = soup.select('li.road')
len(lis)

31

In [32]:
li = lis[0]
name = li.find('a').string.strip()
href = li.find('a')['href']
name, href

('수원중부서', 'https://www.ggpolice.go.kr/swjb/html.do?menu=HC24')

In [33]:
sub_res = requests.get(href)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [37]:
sub_lis = sub_soup.select('.basic_list7 > li')

In [38]:
name = sub_lis[0].get_text().strip()[6:]
name

'수원중부경찰서'

In [39]:
addr = sub_lis[1].get_text().strip()[9:-9]
addr

'경기도 수원시 장안구 정자천로 199'

In [40]:
lines=[]
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        sub_res = requests.get(href)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis = sub_soup.select('.basic_list7 > li')
        name = sub_lis[0].get_text().strip()[6:]
        addr = sub_lis[1].get_text().strip()[9:-9]
        lines.append({'name':name, 'addr':addr})
    except:
        print(index)

6
25


In [41]:
# index6(군포경찰서) 에러(데이터 없음)
lines=[]
for index, li in enumerate(lis):
    href = li.find('a')['href']
    sub_res = requests.get(href)
    sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
    sub_lis = sub_soup.select('.basic_list7 > li')
    if len(sub_lis) == 0:
        continue
    name = sub_lis[0].get_text().strip()[6:]
    addr = sub_lis[1].get_text().strip()[9:-9]
    lines.append({'name':name, 'addr':addr})

In [ ]:
# 정확한 에러 원인을 모를 때 (그냥 넘어가기)
lines=[]
for li in lis:
    try:
        href = li.find('a')['href']
        sub_res = requests.get(href)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        sub_lis = sub_soup.select('.basic_list7 > li')
        name = sub_lis[0].get_text().strip()[6:]
        addr = sub_lis[1].get_text().strip()[9:-9]
        lines.append({'name':name, 'addr':addr})
    except:
        continue

In [43]:
df = pd.DataFrame(lines)
df

,name,addr
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199
1,수원남부경찰서,경기 수원시 영통구 매봉로 52
2,수원서부경찰서,경기 수원시 권선구 서부로 1673
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63
5,과천경찰서,경기 과천시 통영로 20
6,성남수정경찰서,경기 성남시 수정구 성남대로 1259
7,성남중원경찰서,경기 성남시 중원구 금빛로2번길 10
8,분당경찰서,경기 성남시 분당구 정자일로 165
9,부천소사경찰서,경기 부천시 부광로 214


In [44]:
# 중복 데이터 여부 확인
print(df.shape)

(29, 2)


In [45]:
len(df.name.unique())
# 중복 데이터가 있음을 확인

28

In [46]:
# 중복 제거
df.drop_duplicates(subset='name', inplace=True)
df.shape

(28, 2)

In [47]:
import requests, json
from urllib.parse import quote
import folium
import pandas as pd
import numpy as np

In [48]:
with open('../04.지도시각화/data/kakaoapikey.txt') as f:
    kakao_key = f.read()


In [49]:
base_url = "https://dapi.kakao.com/v2/local/search/address.json"
header = {'Authorization': f'KakaoAK {kakao_key}'}
lat_list, lng_list = [],[]
for i in df.index:
    url = f'{base_url}?query={quote(df.addr[i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))

In [50]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,name,addr,위도,경도
0,수원중부경찰서,경기도 수원시 장안구 정자천로 199,37.297932,126.996317
1,수원남부경찰서,경기 수원시 영통구 매봉로 52,37.272104,127.054492
2,수원서부경찰서,경기 수원시 권선구 서부로 1673,37.258740,126.972264
3,안양동안경찰서,경기도 안양시 동안구 동안로159번길 43,37.391209,126.948866
4,안양만안경찰서,경기도 안양시 만안구 냉천로 63,37.387334,126.925974
5,과천경찰서,경기 과천시 통영로 20,37.429358,126.989987
6,성남수정경찰서,경기 성남시 수정구 성남대로 1259,37.442287,127.126662
7,성남중원경찰서,경기 성남시 중원구 금빛로2번길 10,37.442664,127.170371
8,분당경찰서,경기 성남시 분당구 정자일로 165,37.365305,127.105450
9,부천소사경찰서,경기 부천시 부광로 214,37.468723,126.824181


In [53]:
map = folium.Map(location=[df.위도.mean(), df.경도.mean()],zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i],df.경도[i]],
        popup=folium.Popup(df.addr[i], max_width=200),
        tooltip=df.name[i]
    ).add_to(map)

title_html='<h3 align="center" style="font-size:20px">수원소재 경찰서</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map